<a href="https://colab.research.google.com/github/gsbiel/cv-homography/blob/main/custom_cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Computer Vision Assignment - 2**

Students: 
- Gabriel Silva Gaspar; 
- Sara Sampaio Gomes do Nascimento.

In [1]:
!pip install opencv-python==3.4.2.16
!pip install opencv-contrib-python==3.4.2.16

In [6]:
import cv2 as cv
from matplotlib import pyplot as plt
import imutils
import numpy as np

MIN_MATCH_COUNT = 10

In [18]:
# ----------------------------------------------------------------------------------------------------------

"""
  Essa função recebe duas imagens como argumento e retorna os pontos de 
  correspondência entre as duas.
"""

def get_correspondence_points(img1, img2):
  
  # Initiate SIFT detector
  sift = cv.xfeatures2d.SIFT_create()

  # find the keypoints and descriptors with SIFT
  kp1, des1 = sift.detectAndCompute(img1,None)
  kp2, des2 = sift.detectAndCompute(img2,None)
  # FLANN parameters
  # FLANN stands for Fast Library for Approximate Nearest Neighbors. 
  # It contains a collection of algorithms optimized for fast nearest neighbor 
  # search in large datasets and for high dimensional features. 
  # It works faster than BFMatcher for large datasets.
  # The variable index_params specifies the algorithm to be used, its related parameters etc. 
  # For algorithms like SIFT, SURF etc. you can pass following:
  FLANN_INDEX_KDTREE = 1
  index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
  # The variable search_params specifies the number of times the trees in the index should 
  # be recursively traversed. Higher values gives better precision, but also takes more time.
  #search_params = dict(checks=50)   # or pass empty dictionary
  search_params = dict()

  flann = cv.FlannBasedMatcher(index_params,search_params)
  matches = flann.knnMatch(des1,des2,k=2)

  # store all the good matches as per Lowe's ratio test.
  good = []
  for m,n in matches:
      if m.distance < 0.75*n.distance:
          good.append(m)

  if len(good)>MIN_MATCH_COUNT:
      src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
      dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
      M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)
      matchesMask = mask.ravel().tolist()
  else:
    print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )

  return {
      "src_pts":src_pts,
      "dst_pts":dst_pts
  }

# ----------------------------------------------------------------------------------------------------------

"""
Função que teremos que implementar.
M e mask são os dados que ela deve retornar.
"""

def findHomography(src_pts, dst_pts):
  M = np.array([
                [1, 0, 0],
                [0, 1, 1],
                [0, 0, 1]
  ])
  mask = np.array([
                   [0],
                   [1],
                   [1],
                   [0]
                   # ...
  ])
  return [M, mask]

# ----------------------------------------------------------------------------------------------------------

In [20]:
# Carrega imagens para testar a função de homografia
img1 = cv.imread('comicsStarWars01.jpg',0) # queryImage
img2 = cv.imread('comicsStarWars02.jpg',0) # trainImage

# Rotaciona a imagem para ficar melhor a visualização na hora de plotar
img1 = imutils.rotate_bound(img1,90)
img2 = imutils.rotate_bound(img2, 90)

# Obtém pontos de correspondência
correspondence_points = get_correspondence_points(img1, img2)

# Essa é a função que temos que desenvolver!
M, mask = findHomography(correspondence_points["src_pts"], correspondence_points["dst_pts"])